In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import holodeck as holo
from holodeck import detstats
from holodeck.constants import YR, MSOL

import hasasia.skymap as hsky
import hasasia.sensitivity as hsen
import hasasia.sim as hsim

# Glossary:
* F = nfreqs = number of frequencies
* R = nreals = number of strain realizations
* S = nskies = number of single source sky realizations
* L - nloudest = number of loudest single sources at each frequency
# Table of Contents
1. Load or calculate characteristic strain ndarrays, 'hc_bg' of shape (F,R) and 'hc_ss' of shape (F,R,L)
2. Load or calculate ss sky realizations, the single source position, initial gw phase, inclination and polarization
3. Calculate SNR with hasasia
4. Calcualte SNR with holodeck, using 5-dim NDarrays (slow) and Rosado eq.s

# 1.. Load characteristic strain data

In [ ]:
hc_path = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/detstats_functions/npz_arrays/sample_sam_D.npz'  # CHANGE THIS TO YOUR OWN PATH

hcfile = np.load(hc_path)
dur = hcfile['dur']
cad = hcfile['cad']
fobs = hcfile['fobs']
dfobs = hcfile['dfobs']
hc_ss = hcfile['hc_ss']
hc_bg = hcfile['hc_bg']
nfreqs, nreals, nloudest = hc_ss.shape[0], hc_ss.shape[1], hc_ss.shape[2]
print('F=%d, R=%d, L=%d' % (nfreqs, nreals, nloudest)) # 5 frequencies, 1 realization, 1 loudest single source at each frequency

Alternatively, you can calculate this yourself with holodeck, and vary any parameters

In [ ]:
"""

dur = 10.0*YR # choose this, these will determine your frequencies
cad = 0.2*YR # choose this, these will determine your frequencies

fobs_gw_cents = holo.utils.nyquist_freqs(dur,cad)
fobs_gw_edges = holo.utils.nyquist_freqs_edges(dur,cad)
sam = holo.sam.Semi_Analytic_Model() # full SAM 
# sam = holo.sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e11*MSOL, 20), mrat=(1e-3, 1.0, 20), redz=(1e-3, 10.0, 20))  # faster version
hard = holo.hardening.Hard_GW()
fobs=fobs_gw_cents
dfobs = np.diff(fobs_gw_edges)

hc_ss, hc_bg = sam.gwb(fobs_gw_edges, hard=holo.hardening.Hard_GW(), realize=1, loudest = 1, params = False) 
print('F, R, L =', hc_ss.shape)

"""

In [ ]:
fig = holo.plot.plot_bg_ss(fobs, hc_bg, hc_ss)

# 2.. Load (or randomly calculate) skies

In [ ]:
nskies = 1 # number of random single source sky realizations to create

theta_ss = np.random.uniform(0, np.pi, size = nfreqs*nskies*nreals).reshape(nfreqs, nreals, nskies) # shape (F, S, L)
phi_ss = np.random.uniform(0, 2*np.pi, size = hc_ss.size).reshape(hc_ss.shape)
iota_ss = np.random.uniform(0, np.pi, size = hc_ss.size).reshape(hc_ss.shape)
psi_ss = np.random.uniform(0, np.pi, size = hc_ss.size).reshape(hc_ss.shape) 
Phi0_ss = np.random.uniform(0,2*np.pi, size=hc_ss.size).reshape(hc_ss.shape)

# (save these if you want to use the same skies later)
"""
skies_path = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/detstats_functions/npz_arrays/random_binary_angles_D.npz' # CHANGE THIS TO YOUR OWN PATH
np.savez(skies_path,
         theta_ss=theta_ss, phi_ss=phi_ss, iota_ss=iota_ss, psi_ss=psi_ss, Phi0_ss=Phi0_ss) 
"""

Alternatively, load ss sky realizations from a saved file

In [ ]:
"""
skies_path = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/detstats_functions/npz_arrays/random_binary_angles_D.npz' # CHANGE THIS TO YOUR OWN PATH
infile = np.load(skies_path)

theta_ss = infile['theta_ss']
phi_ss = infile['phi_ss']
iota_ss = infile['iota_ss'] # or set to None
psi_ss = infile['psi_ss'] # or set to None
Phi0_ss = infile['Phi0_ss']
"""

# 3.. Build PTA

In [ ]:
npsrs = 40
phis = np.random.uniform(0, 2*np.pi, size = npsrs)
thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
sigmas = np.ones_like(phis)*1e-7

# build sim_pta
pulsars = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigmas,
                    phi=phis, theta=thetas)
# get spectrum for each
spectra = np.empty_like(pulsars, dtype=hsen.Spectrum)
for ii in range(npsrs):
    spectra[ii] = hsen.Spectrum(pulsars[ii], freqs=fobs)
    spectra[ii].NcalInv # calculate inverse noise weighted transmission function


# 4.. Calculate SNR's with hasasia
These calculate the SNR for a source with every theta_ss, phi_ss combination. For sources at our assigned theta's and phi's, take the diagonal of the 2d SNR array

In [ ]:
# calculate strain amplitude from characteristic strain
hs = holo.utils.char_strain_to_strain_amp(hc_ss, fobs, dfobs)

# The following uses the 0th strain realization (R) and 0th loudest sources (L)
# This works for 1 sky realization
# 

# Build hasasia skymap, without iota or psi (inclination and polarization averaged)
skymap1 = hsky.SkySensitivity(list(spectra), theta_gw=theta_ss[:,0,0], 
                                phi_gw=phi_ss[:,0,0],iota=None, psi=None)
# Calculate skymap SNR
snr1 = skymap1.SNR(hs[:,0,0], iota=None, psi=None)

# Try other iota and psi combinations, if you want
# provide iota, not psi
skymap2 = hsky.SkySensitivity(list(spectra), theta_gw=theta_ss[:,0,0], 
                                phi_gw=phi_ss[:,0,0], iota=iota_ss[:,0,0], psi=None)
snr2 = skymap2.SNR(hs[:,0,0], iota=iota_ss[:,0,0], psi=None)

# provide psi, not iota
skymap3 = hsky.SkySensitivity(list(spectra), theta_gw=theta_ss[:,0,0], 
                                phi_gw=phi_ss[:,0,0], iota=None, psi=psi_ss[:,0,0])
snr3 = skymap3.SNR(hs[:,0,0])

This functions, from the old detstats_randomskies.py file, does the same, then takes the diagonal of the 2d hasasia SNR array.

In [ ]:
def snr_skymap(skymap, fobs, dfobs, hc_ss, iota_ss=None, psi_ss=None, 
               debug=True):
    """ Calculate the single source detection probability, and all intermediary steps.
    
    Parameters
    ----------
    skymap : hasasia.skysensitivity.skymap Object
        Skymap from hasasia.
    fobs : (F,) 1Darray of scalars
        Frequency bin centers in Hz.
    dfobs : (F-1,) 1Darray of scalars
        Frequency bin widths in Hz.   
    hc_ss : (F,R,L) NDarray of scalars
        Characteristic strain of the L loudest single sources at 
        each frequency, for R realizations.
    iota_ss : (F,R,L) NDarray or None
        Inclination of each single source with respect to the line of sight.
        NOTE: either `iota_ss` or `psi_ss` can be provided (and not both).
    psi_ss : (F,R,L) NDarray or None
        Polarizationof each single source.
        NOTE: either `iota_ss` or `psi_ss` can be provided (and not both).

    Returns
    -------
    rho_h_ss : (F,R,L) NDarray
        SNR of each single source.

    If neither 'iota_ss' or 'phi_ss' are used, then inclination and polarization 
    averaging is used in hasasia (I think).
    """

    # rho_ss (corresponds to SNR)
    hs = holo.utils.char_strain_to_strain_amp(hc_ss, fobs, dfobs)
    rho_h_ss = np.zeros(hc_ss.shape) # (F,F,R,L)
    # rho_h_ss = np.zeros((hc_ss.shape[0],hc_ss.shape[0],
    #                      hc_ss.shape[1], hc_ss.shape[2])) # (F,F,R,L)
    for rr in range(len(hc_ss[0])):
        for ll in range(len(hc_ss[0,0])):
            if debug: print('hc_ss[:,rr,ll]', hc_ss[:,rr,ll].shape) #, hc_ss[:,rr,ll])
            if(iota_ss is not None): 
                assert (psi_ss is None), "Only one of 'iota_ss' or 'psi_ss' should be provided."    
                iota=iota_ss[:,rr,ll] 
            else: 
                iota = None
            if(psi_ss is not None): 
                psi = psi_ss[:,rr,ll] # otherwise 
            else: 
                psi = None
            if debug: print('iota =', iota, 'psi =', psi)
            rho = skymap.SNR(hs[:,rr,ll], iota=iota, psi=psi) # calcutes SNR for every phi theta combo
            rho = np.diagonal(rho) # SNR only for the corresponding phis and thetas
            try:
                rho_h_ss[:,rr,ll]  = rho
                if (rr==0) and (ll==0):
                    if debug: print('rho', rho.shape) #, rho)
            except Exception:
                if debug: print('rho', rho.shape) #, rho)
                if debug: print("'rho_h_ss[:,rr,ll]  = rho' failed")
                raise
                # return rho_h_ss
    
    return rho_h_ss

In [ ]:
snr4 = snr_skymap(skymap1, fobs, dfobs, hc_ss, iota_ss=None, psi_ss=None, debug=True)

assert np.all(np.diag(snr1) == snr4.squeeze()),  'Diagonal of snr1 should equal snr4, something might be wrong!'
for ii in range(len(snr4)):
    assert snr4[ii] == snr1[ii,ii], 'Diagonal of snr1 should equal snr4, something might be wrong!'

# 5.. Calculate SNR's with holodeck
 Based on Rosado et al. 2015

Here are the individual steps, if you want to look at intermediate variables

In [ ]:
 # unitary vectors
m_hat = detstats._m_unitary_vector(theta_ss, phi_ss, psi_ss) # (3,F,S,L)
print(m_hat.shape)
n_hat = detstats._n_unitary_vector(theta_ss, phi_ss, psi_ss) # (3,F,S,L)
Omega_hat = detstats._Omega_unitary_vector(theta_ss, phi_ss) # (3,F,S,L)
pi_hat = detstats._pi_unitary_vector(phis, thetas) # (3,P)

# antenna pattern functions
F_iplus, F_icross = detstats._antenna_pattern_functions(m_hat, n_hat, Omega_hat, 
                                                pi_hat) # (P,F,S,L)
print(F_iplus.shape)

# noise spectral density
S_i = detstats._total_noise(cad, sigmas, hc_ss, hc_bg, fobs)

# amplitudw
amp = detstats._amplitude(hc_ss, fobs, 
                    dfobs) # (F,R,L)
print(amp.shape)

snr_ss_5d = detstats._snr_ss_5dim(amp, F_iplus, F_icross, iota_ss, dur, Phi0_ss, S_i, fobs) # (F,R,S,L)
print(snr_ss_5d.shape)

The detect_ss_pta() function does all this, and also calculates detection probabilities. 

The only variable that may need changing is Fe_bar_guess. This is used by sympy.nsolve to solve for the threshold Fe statistic. If you use larger F or L, try 15 or 20. 

In [ ]:
gamma_ss, snr_ss, gamma_ssi = detstats.detect_ss_pta(pulsars, cad, dur, fobs, dfobs, hc_ss, hc_bg,
                                                     theta_ss, phi_ss, Phi0_ss, iota_ss, psi_ss,
                                                     snr_cython=False,
                                                     gamma_cython=False, 
                                                     Fe_bar_guess=10, # you may need to adjust Fe_bar_guess
                                                     ret_snr=True)

# Plot/Compare

I'll leave most of this to you, but here's one plot to get ya started!

In [ ]:
fig, ax = holo.plot.figax(xlabel = 'Frequency, $f$ (1/yr',
                          ylabel = 'Single Source SNR')

xx = fobs*YR

ax.scatter(xx, snr4, label='skymap SNR with iota=psi=None', alpha=0.5)
ax.scatter(xx, np.diag(snr2), label='skymap SNR with random iota, psi=None', alpha=0.5)
ax.scatter(xx, np.diag(snr3), label='skymap SNR with random iota, psi=None', alpha=0.5)
ax.scatter(xx, snr_ss, label='Rosado SNR with random iota, psi', alpha=0.5)

ax.legend()